In [1]:
import plotly.graph_objects as go
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/titanic.csv')

In [3]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [4]:
df1 = df.groupby(['Pclass', 'Sex'])['Name'].count().reset_index()
df1.columns = ['source', 'target', 'value']
df1['source'] = df1.source.map({1: 'Pclass1', 2: 'Pclass2', 3: 'Pclass3'})
df2 = df.groupby(['Sex', 'Survived'])['Name'].count().reset_index()
df2.columns = ['source', 'target', 'value']
df2['target'] = df2.target.map({1: 'Survived', 0: 'Died'})
links = pd.concat([df1, df2], axis=0)

In [5]:
links

,source,target,value
0,Pclass1,female,9
1,Pclass1,male,21
2,Pclass2,female,12
3,Pclass2,male,18
4,Pclass3,female,35
5,Pclass3,male,61
0,female,Died,16
1,female,Survived,40
2,male,Died,86
3,male,Survived,14


In [6]:
unique_source_target = list(pd.unique(links[['source', 'target']].values.ravel('K')))

In [7]:
unique_source_target

['Pclass1', 'Pclass2', 'Pclass3', 'female', 'male', 'Died', 'Survived']

In [8]:
mapping_dict = {k: v for v, k in enumerate(unique_source_target)}

In [9]:
mapping_dict

{'Pclass1': 0,
 'Pclass2': 1,
 'Pclass3': 2,
 'female': 3,
 'male': 4,
 'Died': 5,
 'Survived': 6}

In [10]:
links['source'] = links['source'].map(mapping_dict)
links['target'] = links['target'].map(mapping_dict)

In [11]:
links

,source,target,value
0,0,3,9
1,0,4,21
2,1,3,12
3,1,4,18
4,2,3,35
5,2,4,61
0,3,5,16
1,3,6,40
2,4,5,86
3,4,6,14


In [12]:
links_dict = links.to_dict(orient='list')

In [13]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = unique_source_target,
      color = "blue"
    ),
    link = dict(
      source = links_dict["source"],
      target = links_dict["target"],
      value = links_dict["value"]
  ))])

In [14]:
fig.update_layout(title_text="Titanic Survival Sankey Diagram", font_size=10)
fig.show()